### Homework 7, Computational Economics, ECON-GA 3002-12

**{Name, NYUID, Email}**: Arnav Sood, N11193569, asood@nyu.edu

> **Exercise 1**: Let an equilibrium asset price function $p(x)$ be characterized recursively by $p(x) = \beta \sum_{y \in S} \big\{d(y) + p(y)\big\}q(x,y)$ for a countable state space $S$. Assuming that $S$ is finite, write a function that takes a specification of the primitives and iterates to convergence on $p(x)$, in the sense of the Banach contraction mapping theorem.

> Write a function to compute the $l_\infty$ norm of a vector.

> **Note:** The wrapper function provided is at the end of this section. We would ordinarily include all of these auxiliary functions as parts of a closure. But, because that requires one block of code, I decided to break it up for readability.

In [1]:
function computeNorm(priceVec)
    return maximum(abs(priceVec))
end

computeNorm (generic function with 1 method)

> Write a function to take the primitives (a price guess, dividend function, discount rate, and transition probability matrix) and iterate forward.

In [2]:
function iterate(priceVec,dividendFunc,beta,transitionMatrix)
    newPriceVec = zeros(length(priceVec))
    for originState in 1:length(priceVec)
        sum=0
        for targetState in 1:length(priceVec)
            sum+=(dividendFunc[targetState] + priceVec[targetState])*transitionMatrix[originState,targetState]
        end
        newPriceVec[originState] = beta*abs(sum)
    end
    return newPriceVec
end

iterate (generic function with 1 method)

> Write a function to iterate on the $l_{\infty}$ norm until it reaches a specified bound.

In [3]:
function approximate(bound,initialGuess,dividendFunc,beta,transitionMatrix)
    diffNorm = computeNorm(iterate(initialGuess,dividendFunc,beta,transitionMatrix)-initialGuess)
    while diffNorm >= bound
        diffNorm = computeNorm(iterate(initialGuess,dividendFunc,beta,transitionMatrix)-initialGuess)
        initialGuess = iterate(initialGuess,dividendFunc,beta,transitionMatrix)
    end
    return initialGuess
end

approximate (generic function with 1 method)

> Define the first set of primitives, and call for them.

In [4]:
const dividendFunc = [1 0 0]
const bound = 0.0001
const beta = 0.98
global initialGuess = zeros(length(dividendFunc))+1//length(dividendFunc)
const transitionMatrix1 = [0.971 0.029 0; 0.145 0.778 0.077;0 0.508 0.492]

println(approximate(bound,initialGuess,dividendFunc,beta,transitionMatrix1))

[40.75495110518733,35.956059179311126,34.56717911413835]


>Now, alter $q$ and try it again.

In [5]:
initialGuess = zeros(length(dividendFunc))+1//length(dividendFunc)
const transitionMatrix2 = [0.871 0.029 0.1; 0.145 0.778 0.077; 0 0.508 0.492]
println(approximate(bound,initialGuess,dividendFunc,beta,transitionMatrix2))

[23.496270507071706,21.09396688430362,20.279088501031502]


> **Intrepretation:** We can see that, uniformly, asset prices have decreased. This is because the stationary distribution has shifted to spending less time in period `NG` (the period which pays dividends). Moreover, the way this shift was effected (spend more time in `SR`, the "worst" state, as the expected time to return to `NG` from `SR` is longest) is the most depressing on asset values. 

> **Exercise 2:** Admit heterogeneous beliefs: that is, $p(x) = \max\limits_{i \in \{1, 2\}} \big\{\sum_{y\in S}\big[d(y) + p(y)\big]q_i(x,y)\big\}$ for two different belief functions $q_1, q_2$. Give conditions under which this equation has a unique solution in $b\mathbb{R}^S$, and prove those conditions sufficient.

The conditions are the same as in the homogeneous case: namely, $d$ bounded.


Loosely speaking, our goal is to show that the $\max$ operator --- combined with the inequality given --- reduces the problem to the single-variable case as far as existence and uniqueness is concerned, and to prove the inequality.

**Inequality Proof**: Consider the $i$ that yields $a_{\max}$. We have $|a_{\max} - b_{\max}| > |a_{\max} - (b_{\max} - \epsilon)| = |a_{\max} - b_{\max} + \epsilon|$.  If $a_{\max} - b_{\max}$ is positive or 0, that's clearly false. If it's negative, pick the $i$ that yields $b_{\max}$ to obtain $|a_{\max} - b_{\max}| < |(a_{\max} - \epsilon) - b_{\max}| = |a_{\max} - b_{\max} - \epsilon|$, which is again clearly false. So, the inequality holds.

**Fixed-Point Proof**:

For boundendess, recall:
\begin{equation}
Tp(x) = \beta \max\limits_{i \in \{1, 2\}}\big\{\sum\limits_{y \in S}[d(y) + p(y)]q_i(x,y)\big\}
\end{equation}

This implies:

\begin{equation}
|Tp(x)| = |\beta \max\limits_{i \in \{1, 2\}}\big\{\sum\limits_{y \in S}[d(y) + p(y)]q_i(x,y)\big\}|
\end{equation}

Using the definition of absolute value:

\begin{equation}
|Tp(x)| \leq \beta \max\limits_{i \in \{1, 2\}}\big\{\sum\limits_{y \in S}[d(y) + p(y)]q_i(x,y)\big\}
\end{equation}

Now, the expression inside the $\max$ operator is equal to either the value given by $q_1$ or that given by $q_2$ (or both, in the indifference case). The important part is that it's given by one of them. Call this $q_{\max}$, and write:

\begin{equation}
|Tp(x)| \leq \beta \sum\limits_{y \in S}[d(y) + p(y)]q_{\max}(x,y)
\end{equation}

We know that this implies:

\begin{equation}
|Tp(x)| \leq \beta \sum\limits_{y \in S}[||d|| + ||p||]q_{\max}(x,y) = \beta(||d|| + ||p||)
\end{equation}

The sum works as we know that $q$ sums to 1 (i.e., it is drawn from an infinite discrete distribution, like the Poisson). So, boundedness of the heterogeneous beliefs case comes from boundedness of the homogeneous beliefs case, and since $x$ was arbitrary it applies to the entire function.

The same kind of thing holds for demonstrating uniform contraction. Let $p, p'$ be initial guesses on the price, in $b\mathbb{R}^S$. Then:

\begin{equation}
|Tp(x) - Tp'(x)| = \beta|\max\limits_{i \in \{1, 2\}}\sum_{y \in S}[d(y) + p(y)]q_i(x,y)| - |\max\limits_{i \in \{1, 2\}}\sum_{y \in S}[d(y) + p(y)]q_i(x,y)|
\end{equation}

But, using the inequality, we know this is less than $|T(p-p')(x)|$, or $\max\limits_{i \in \{1, 2\}}|\sum_{y \in S}[d(y) + p(y) - p'(y)]q_i(x,y)|$. As the difference of two bounded functions is itself bounded, we know that $|(p-p')(y)| = |p(y) - p'(y)|$ is bounded. Further, we know there is a maximizing $q_{\max}$(i.e., that the max operator will return a result), as we already showed it maps bounded functions to bounded functions. Therefore, this reduces exactly to the homogeneous belief problem for $p_{\text{guess}} = p(x) - p'(x)$ for initial guesses $p, p'$, and again we can invoke the arbitrariness of $x$ to show that uniform contraction holds for this operator applied to any guesses in $b\mathbb{R}^S$ at any argument in the state space.




> **Exercise 3:** Write another function to test convergence on the heterogeneous beliefs case, using the primitives specified in Exercise 1, and the belief functions (transition probabilities) expressed in Hamilton.

> In other words, we must alter our final wrapper, `approximate`, to take two stochastic matrices in its arguments. We must also alter our `iterate` function, which performs $T: b\mathbb{R}^S \to b\mathbb{R}^S$, to perform the heterogeneous beliefs case.


In [6]:
function iterate2(priceVec,dividendFunc,beta,transitionMatrix1,transitionMatrix2)
        candidate1 = iterate(priceVec,dividendFunc,beta,transitionMatrix1)
        candidate2 = iterate(priceVec,dividendFunc,beta,transitionMatrix2)
        for element in 1:length(priceVec)
                candidate1[element] = max(candidate1[element],candidate2[element])
        end
return candidate1
end

iterate2 (generic function with 1 method)

In [7]:
function approximate2(bound,initialGuess,dividendFunc,beta,transitionMatrix1,transitionMatrix2)
    iterateNorm = computeNorm(iterate2(initialGuess,dividendFunc,beta,transitionMatrix1,transitionMatrix2)-initialGuess)
    while iterateNorm >= bound
        iterateNorm = computeNorm(iterate2(initialGuess,dividendFunc,beta,transitionMatrix1,transitionMatrix2)-initialGuess)
        initialGuess = iterate2(initialGuess,dividendFunc,beta,transitionMatrix1,transitionMatrix2)
    end
    return initialGuess
end

approximate2 (generic function with 1 method)

In [8]:
initialGuess = zeros(length(dividendFunc))+1//length(dividendFunc)
println(approximate2(bound,initialGuess,dividendFunc,beta,transitionMatrix1,transitionMatrix2))

[40.75495110518733,35.956059179311126,34.56717911413835]
